In [1]:
import json
import collections
import subprocess

def prettify(atom):

    s = atom['predicate']
    if 'terms' in atom:
        s += '('
        ts = [prettify(t) for t in atom['terms']]
        s += ','.join(ts)
        s += ')'
    return s

  
def parse_json_result(out):
    """Parse the provided JSON text and extract a dict
    representing the predicates described in the first solver result."""
    result = json.loads(out)
    assert len(result['Call']) > 0
    assert len(result['Call'][0]['Witnesses']) > 0
    all_preds = []
    ids = range(len(result['Call'][0]['Witnesses']))
    
    witness = result['Call'][0]['Witnesses'][0]['Value']

    class identitydefaultdict(collections.defaultdict):
        def __missing__(self, key):
            return key

    preds = collections.defaultdict(list)
    env = identitydefaultdict()

    for atom in witness:
        parsed,dummy = parse_terms(atom)
        preds[parsed[0]['predicate']].append(parsed)
    return preds

def solve(args):
    """Run clingo with the provided argument list and return the parsed JSON result."""

    args = ['clingo','--outf=2'] + args
    clingo = subprocess.Popen(
        ' '.join(args),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        shell=True
        )
    out, err = clingo.communicate()
            
    return parse_json_result(out)

def parse_terms(arguments):
    terms = []
    while len(arguments) > 0:
        l_paren = arguments.find('(')
        r_paren = arguments.find(')')
        comma = arguments.find(',')
        if l_paren < 0:
            l_paren = len(arguments)-1
        if r_paren < 0:
            r_paren = len(arguments)-1
        if comma < 0:
            comma = len(arguments)-1
        next = min(l_paren,r_paren,comma)
        next_c = arguments[next]
        if next_c == '(':
        
            pred = arguments[:next]
            sub_terms, arguments = parse_terms(arguments[next+1:]) 
            terms.append({'predicate':pred,'terms':sub_terms})
        elif next_c == ')':
            pred = arguments[:next]
            if pred != '':
                terms.append({'predicate':arguments[:next]})
            arguments = arguments[next+1:]
            return terms,arguments
        elif next_c == ',':
            pred = arguments[:next]
            if pred != '':
                terms.append({'predicate':arguments[:next]})
            arguments = arguments[next+1:]
        else:
            terms.append({'predicate':arguments})
            arguments = ''
    return terms, ''
   


In [2]:
filename = 'pong.lp'

rules = open(filename,'rb').read().replace(' ','').replace('\n','').split('.')[:-1]
rules = [parse_terms(rule)[0][0] for rule in rules]
facts = []
types = {}
for rule in rules:
    print rule
    if rule['predicate'] == 'type':
        print rule
        types[rule['terms'][1]['predicate']] = rule['terms'][0]['predicate']
    else: 
        facts.append(rule)
output_rules = [prettify(rule) for rule in facts]
print types

{'predicate': 'type', 'terms': [{'predicate': 'boolean'}, {'predicate': 'boolean'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boolean'}, {'predicate': 'boolean'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boolean'}, {'predicate': 'false'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boolean'}, {'predicate': 'false'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boolean'}, {'predicate': 'true'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boolean'}, {'predicate': 'true'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boundary'}, {'predicate': 'boundary'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boundary'}, {'predicate': 'boundary'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boundary'}, {'predicate': 'screen_left'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boundary'}, {'predicate': 'screen_left'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boundary'}, {'predicate': 'screen_right'}]}
{'predicate': 'type', 'terms': [{'predicate': 'boundary

In [3]:

def has_term(rule,term):
    
    if 'terms' in rule:
        
        for rule_term in rule['terms']:
                if has_term(rule_term,term):
                    return True
        return False
    elif rule['predicate'] == term:
        return True
    else:
        return False
def get_terms(rule):
    if 'terms' in rule:
        terms = []
        for rule_term in rule['terms']:
            terms += get_terms(rule_term)
        return terms
    else:
        return [rule['predicate']]

In [12]:
import random
import sys

def generalize(fact,probability=0.5):   
        
    if 'terms' in fact:
        terms = []
        if len(fact['terms']) == 1 and random.random()*2 < probability:
            return {'predicate':fact['predicate'],
                'terms':[{'predicate':
                        fact['predicate'].upper()}]}
        for fact_term in fact['terms']:
            terms.append(generalize(fact_term,probability))
        return {'predicate':fact['predicate'],
                'terms':terms}
    else:
        pred = str(fact['predicate'])
        if random.random() < probability:
            pred = pred.upper()
        return {'predicate':pred}
    
def replace(fact,source,target):
    if 'terms' in fact:
        terms = []
        for fact_term in fact['terms']:
            terms.append(replace(fact_term,source,target))
        return {'predicate':fact['predicate'],
                'terms':terms}
    else:
        pred = fact['predicate']
        if pred == source:
            pred = target
        return {'predicate':pred}
    
def generate_rule(target_form,facts,predecessor=None):
    if predecessor:
        pass
    else:
        number_of_rules = random.randrange(2,3)
        random.shuffle(facts)
        facts_to_use = [generalize(fact,0.8) for fact in facts[:number_of_rules]]
        uniques = set()
        
        for fact_id, fact in enumerate(facts_to_use):
            terms = set(get_terms(fact))
            uniques |= set([(fact_id,term) for term in terms])
            
        by_type = {}
        for u in uniques:
            t = types[u[1].lower()]
            if t not in by_type:
                by_type[t] = []
            by_type[t].append(u)
            
        
        unique_mapping = {}
        can_be_used_by_type = {}
        for u in uniques:
            t = types[u[1].lower()]
            if t not in can_be_used_by_type:
                can_be_used_by_type[t] = []
            unique_mapping[u] = 'V{}{}'.format(t,random.randrange(len(by_type[t])))
            can_be_used_by_type[t].append(unique_mapping[u])
            
        by_rule_mapping = {}
        
        for u,m in unique_mapping.items():
            if u[0] not in by_rule_mapping:
                by_rule_mapping[u[0]] = {}
            by_rule_mapping[u[0]][u[1]] = m
            
            
            
        final_facts = [] 
        
        for fact_id, fact in enumerate(facts_to_use):
            
            terms = set(get_terms(fact))
            for term in terms:
                fact = replace(fact,term,by_rule_mapping[fact_id][term])
            final_facts.append(fact)
        
        terms = list(set(get_terms(target_form)))
        for term in terms:
            if term not in can_be_used_by_type:
                return (None,None)
            target_form = replace(target_form,term,random.choice(can_be_used_by_type[term]))
        return target_form,final_facts
        
target_rule  = {'predicate':'player_controls','terms':[{'predicate':'entity'}]}



positives = [{'predicate':'player_controls','terms':[{'predicate':'paddle_player'}]}]
positives = [prettify(f) for f in positives]
for ii in range(1000):
    target_head = None
    
    while target_head == None:
        target_head, target_body = generate_rule(target_rule,facts)
        if target_head != None:
            with open('temp.lp','wb') as outfile:
                outfile.write('.\n'.join(output_rules) + '.\n')
                outfile.write(prettify(target_head) + ':-' + ','.join([prettify(body) for body in target_body]) + '.\n')
                outfile.write('#show player_controls/1.')
            solved = solve(['temp.lp'])
            is_good = True
            found = []
            for t in solved:
                for tt in solved[t]:
                    for ttt in tt:
                        if prettify(ttt) in positives:
                            found.append(prettify(ttt))
                        else:
                            is_good = False
                            break
                    if not is_good:
                        break
                if not is_good:
                    break
            if is_good and len(found) == len(positives):
                print prettify(target_head) + ':-' + ','.join([prettify(body) for body in target_body]) + '.\n'
#rules_ = solve(['temp.lp'])


player_controls(Ventity0):-precondition(control_event(player_input(Vkey0,Vpress_type0)),Voutcome1),result(Voutcome1,moves(Ventity0,Vdirection0,Vscalar0)).

player_controls(Ventity0):-precondition(control_event(Vcontrol_event0),Voutcome0),result(Voutcome0,moves(Ventity0,Vdirection0,Vscalar0)).

player_controls(Ventity0):-precondition(control_event(Vcontrol_event0),Voutcome1),result(Voutcome1,moves(Ventity0,Vdirection0,Vscalar0)).

player_controls(Ventity0):-result(Voutcome0,moves(Ventity0,Vdirection0,Vscalar0)),precondition(control_event(player_input(Vkey0,Vpress_type0)),Voutcome0).

player_controls(Ventity0):-precondition(control_event(player_input(Vkey0,Vpress_type0)),Voutcome0),result(Voutcome0,moves(Ventity0,Vdirection0,Vscalar0)).

player_controls(Ventity0):-result(Voutcome0,moves(Ventity0,Vdirection0,Vscalar0)),precondition(control_event(player_input(Vkey0,Vpress_type0)),Voutcome0).

player_controls(Ventity0):-precondition(control_event(player_input(Vkey0,Vpress_type0)),Voutcome1)